# Image Analysis with Python

In this workshop, we're going to load and explore some satellite imagery, then calculate some indices such as NDVI.
By the end, you will have learned a bit about the NetCDF data format, how to load and visualise gridded data, and can say that you've worked with "Big Data"!

Below, I've written some demonstration code to:

1. load a MODIS composite image of Australia
2. view and understand the metadata
3. view the image and draw maps
4. calculate and view NDVI, GCC, and EVI

The MatLab workshop this week includes some threshold and classification analysis - we will spend that time on understanding more complicated data formats, and look at classification next week.


## Loading Big Data

First, let's import the libraries (packages of code) that we want to use for this task.  `numpy` is the foundation of scientific Python, by supporting very fast numerical operations on arrays.  `matplotlib`, well, is a matlab-style plotting library; we're importing `seaborn` because it makes the default styles much nicer (and has nice statistical graphs, but that's a topic for an optional session later).  `xarray` [(docs)](http://xarray.pydata.org) is the nicest way to work with labelled multidimensional data - specialised, but indispensible for us.

In [ ]:
# NumPy for arrays, and Xarray for gridded geospatial datasets
import numpy as np
import xarray as xr

Next, we'll open our dataset for this workshop - a MODIS composite covering all of Australia.

For the interested, here's how I found the data - note that **you won't need to do this for the course**; I'll be happy to help you find data if you need it for your research paper.  Identifying what data you want or need and where it can be found is often one of the most challenging parts of an analysis.  In this case I visited the TERN (terrestrial ecosystem research network) Auscover website, and searched for "MODIS mosaic" - finding [this page](http://data.auscover.org.au/xwiki/bin/view/Product+pages/LPDAAC+Mosaics+MxD09+CMAR).  From there I clicked through NetCDF and Australian mosaics to find [this listing](http://data.auscover.org.au/thredds/catalog/auscover/lpdaac-csiro/c5/v2-nc4/aust/catalog.html).  I chose "Nadir BRDF-Adjusted Reflectance 16-Day 500m - Combined" because it adjusts for atmospheric effects, view angle, illumination, etc; and taking the best quality pixel in 16 days avoids most problems with clouds etc (at least in Australia!).

`xarray`, the tool we will use for NetCDF data, can load data from a URL as well as a file.  If this data is provided via OpeNDAP (**Ope**n **N**etwork **D**ata **A**ccess **P**rotocol), `xarray` will automatically avoid downloading data until you need it - so opening very large collections of files only transfers a little metadata, and taking subsets is usually quite efficient.  We'll therefore avoid downloading anything manually, and just use the OpeNDAP link for a recent mosaic.  (There's also multi-file support, but more on that later)

In [ ]:
# Specify the filename and url on two lines for readability
# Note that there are tools to discover these links automatically, but we'll do it manually for now
file_name = 'MCD43A4.2017.073.aust.005.nadir_brdf_adjusted_reflectance.nc'
url = 'http://data.auscover.org.au/thredds/dodsC/auscover/lpdaac-csiro/c5/v2-nc4/aust/MCD43A4.005/2017.03.14/'

# Open the dataset and see what's inside
ds = xr.open_dataset(url + file_name)
ds

What does all of this mean?  In order:

- `<xarray.Dataset>` means that this represents... an [xarray dataset](http://xarray.pydata.org/en/stable/data-structures.html#dataset): "a dict-like container of labeled arrays (DataArray objects) with aligned dimensions. It is designed as an in-memory representation of the data model from the netCDF file format."

- `Dimensions:` lists the dimensions of this data in space and time, and the size of the data in each dimension.  Latitude, longitude, and time are pretty obvious, but what could `nv` represent?  We'll see in the next point!

- `Coordinates:` contains a a list of "coordinate arrays", which tell us the location that each index in our arrays of data corresponds to in a real coordinate system.  In a NetCDF file these are stored seperately to the dimensions, by `xarray` is smart enough to handle this for us; the dimension or dimensions of a data array are shown in brackets, then the type, then the first few values.  For example, time is measured along the time dimension (yep!), in a 64-bit date-and-time stamp with nanosecond precision (!!), and the first value is March 14th, 2017.

- `Dimensions without coordinates: nv`... huh?  If you look down at the data variables section, the `time_bounds` array has a time and a `nv` dimension.  Looking at `ds.nv` in a new cell shows that it's equal to `[0, 1]` - so it looks like this is used to describe the edges of the pixels.  It's useful to recognise this when you see it, but we won't be using pixel boundaries in this course!

- `Data variables:` is where the fun really starts: they're the variables with data in them!  (did you guess?)

  - `crs` is the coordinate reference system.  Critically important for changing the data projection, but we'll just lat/long coordinates and don't care today.
  - time_bounds, lat_bounds, and lon_bounds describe pixel boundaries, as mentioned above.
  - `nbar_0620_0670nm` through to `nbar_2105_2155nm` are the surface reflectance data - the whole reason we got this file!
  - typical_mask, quality, and snow all describe quality issues with the data.  This is something you should usually think about and handle carefully to ensure that your results are valid, but is less important when you're just learning to open files and explore the data.
    
  Note that where the coordinates entries showed their first few values, the data variables just show `...`.  Because we are loading over the internet, the data is not actually downloaded until we attempt to use it.  This makes many operations (eg subsetting) much faster, because we can avoid downloading anything we won't actually use.  This is convenient on a 400MB file, and crucial when working with larger collections (such as all such files through time) which may be many gigabytes or even terabytes.  The idea that you can work with collections of files is also the reason to include a time dimension with only one step - it's actually just one step in this file, and many time steps across all the files.
  
- Finally, `Attributes:` lists all the other metadata of the file in general.  Author, contact, provenance, description; it's all there.  You can inspect `ds.attrs` in a new cell to see the full text of each, since it's truncated to fit on one line in this summary.  The `Conventions`, `standard_name_vocabulary`, and `keywords_vocabulary` indicate that this file uses standardised names (and which version of each standard), which enables much more useful automatic analysis in specialised programs.

Let's have a quick look at some of these data and metadata...

In [ ]:
ds.attrs['summary']

In [ ]:
# This is the coordinate array for time.
# Note that arrays have their own metadata, just like the full dataset
ds.time

In [ ]:
# Let's check a more complicated data array
ds.nbar_0459_0479nm

The `_ChunkSizes` attribute can be very important for performance with large data or operations.  Check out [this blog post from a NetCDF developer](https://www.unidata.ucar.edu/blogs/developer/entry/chunking_data_why_it_matters) if you're interested.


## Making Maps

Visualising our data can be a good starting point for an analysis, and a beautiful and informative map might be your most important final output too!

Python has excellent graphing capabilities, in two different styles:

- with low level control, where you describe *how to draw your figure*.  Anything is possible at this level, but drawing something elegant is usually difficult and always time consuming.
- with a high level interface that lets you *describe what you want*, and leave the details to code written by experts in visualisation (using the low-level controls!).  Taking the high-level option is always a better choice if you are not drawing heavily customised plots.

The first step is usually to clean up our data a bit.  I'm going to subset it, drop the variables we're not interested in, and finally load the part we'll use into memory (ie off the supercomputer).  This makes plotting a second map much faster.

In [ ]:
# Discard data vars whose name does not start with "nbar_"
vis = ds.drop([n for n in ds.data_vars if not n.startswith('nbar_')])
vis.attrs = {}  # Discard attributes metadata

# This indicates "no start point, no end point, one pixel per 25" - plenty for example charts
zoom = slice(None, None, 25)
# Select every 25th point of lat and lon, and the 0th (first) time step
vis = vis.sel(longitude=zoom, latitude=zoom).isel(time=0)

# And now download this subset, since we'll be reusing it - it'll take a minute or so
vis.load()
vis

See that the data varaibles now have some values shown before the `...`?  The're `nan` for 'not a number', because the top-left corner of our image is in the ocean, but they've been loaded onto this computer.

OK, with our data in order - finally! - it's time to start the visualisation.  We'll begin with `matplotlib` ([gallery](https://matplotlib.org/gallery.html), [reference](http://matplotlib.org/api/pyplot_api.html)), the low-level tool that almost everything else is built on top of.

In [ ]:
# Load the graphing library and show images inline, instead of saving to a file
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
plt.imshow(vis.nbar_0459_0479nm)

[`plt.imshow`](http://matplotlib.org/api/pyplot_api.html#matplotlib.pyplot.imshow) shows an array as an image.  It's even smart enough to use a colour map instead of just displaying in greyscale.  However, the generality of this function (it works for any array at all!) means that it doesn't draw very good maps, even if you use the customisation options (see link).

- The axis labels are using pixel indices, not the lat/lon coordinates
- We don't have axis labels, a title, a colour bar, etc.
- The colour range is really compressed due to outliers

Some of this can be fixed, though it gets pretty tedious:

In [ ]:
plt.title('Title for my map')
plt.xlabel('Longitude (in degrees)')
plt.xticks(range(0, len(vis.longitude) - 1, 50),        # Where to put the labels - every 50th pixel
           [int(v) for v in vis.longitude.data[::50]])  # What the labels are - every 50th lon, as an integer
plt.ylabel('Latitude (in pixels, so not really)')
plt.imshow(vis.nbar_0459_0479nm, vmin=0, vmax=0.15)
plt.colorbar()

This is tedious, but worse - it's also error-prone, subjective, and hard to adjust later.  Shouldn't the colour bar have an open top?  What if we reuse the code for a different band, and forget to update the title?  And the x-axis labels won't work for higher resolution data, plus it's pretty odd to mark every five-or-six degrees!

`xarray` to the rescue!  Because our `dataset` and `dataarray` objects know about their own coordinates and metadata, the `xarray` developers (or anyone interested, like me) can write customised presentation logic.

For example, every data array has a customised version of the `pyplot` functions as it's `.plot` attribute:

In [ ]:
vis.nbar_0459_0479nm.plot.imshow()

Wow!  The axes are sensibly labelled, there's a colourbar (with the thing it measures), and our title has been automatically filled in.

But what if we want to ignore the outliers again, or use a different title?  It's still user-customisable - it simply starts with better defaults!

In [ ]:
vis.nbar_0459_0479nm.plot.imshow(robust=True)  # will ignore the 0:2 and 98:100 percentiles of any data
plt.title('See, we can still use the low-level tools')

Pretty cool!  If you want to learn more about Python, I still suggest reading [*Think Python*](http://greenteapress.com/wp/think-python-2e/) - as a textbook, it goes from bare basics through to implementing your own objects.  Read it, and you'll understand everything I've done here - and have gained very useful programming skills! 

Using the dataset directly also lets us draw some plots that would be very difficult to make otherwise:

In [ ]:
# We prepare for this one by stacking our 2D arrays into a new 3D array,
# along a new dimension called "band"
arr = vis.to_array(dim='band')
arr

In [ ]:
# Now we're dealing with a 'cube' (OK, rectangular prism) of data.
# And... we can draw a faceted plot by slicing along an axis:
arr.plot.imshow(robust=True, col='band', col_wrap=4)

## Calculating Indices

Calculating per-pixel indices such as NDVI (normalised vegetation difference index), EVI (enhanced vegetation index), or GCC (green chromatic coordinate) is quite simple with Python.  Remember that arithimetic with arrays operates on each element - `xarray` arrays work the same way (also aligning coordinates if required) so you can simply follow the formulae in your reading.

In [ ]:
# We'll give the bands english names instead of using nanometer measurements
blue = vis.nbar_0459_0479nm
green = vis.nbar_0545_0565nm
red = vis.nbar_0620_0670nm
nir = vis.nbar_0841_0876nm

In [ ]:
# It's a pretty simple calculation!
NDVI = (nir - red) / (nir + red)
# What happens if you don't exclude negative numbers?  Why might this happen?
NDVI.plot.imshow(robust=True, vmin=0)

In [ ]:
GCC = green / (red + green + blue)
GCC.plot.imshow(robust=True)

NDVI and GCC are very easy to calculate, and give generally similar results (because, of course, they're both correlated with presence of green vegetation).  What about [EVI](https://en.wikipedia.org/wiki/Enhanced_vegetation_index)?

If it's "enhanced", it must be better:

In [ ]:
# Using the MODIS-EVI coefficients
L, C1, C2, G = 1, 6, 7.5, 2.5
EVI = G * ((nir - red) / (nir + C1 * red - C2 * blue + L))
EVI.plot.imshow(robust=True, vmin=0)

Notice how NDVI and EVI are more similar than GCC?  Your readings should describe the motivation behind each - and why anyone would want to use GCC.

Finally, note that calculating standard indices like this is purely a learning excercise.  If you wanted to *use* the indices, you can get pre-calculated versions as easily as reflectance, and the official products have much better quality control than us - and squeeze out every bit of spatial resolution possible.


## Summary and Research Ideas

This Tutorial showed how to open satellite data in NetCDF format, how 
to display them as bands or as colour composites, and how to calculate indices.

You could develop a research paper using reflectances to

* Test alternative band indices to map features of interest;
* Collect ground-truth data to map using a supervised classification method

Data you could use include MODIS data (for this or another date) or airborne 
hyperspectral imagery used in the MatLab tutorial. Alternatively, you could use Landsat 
imagery of any part of Australia (to be introduced in one of the following tutorials), 
or you could use hyperspectral imagery the ACT (available from Marta Yebra).

Some features or surfaces of interests could be a particular vegetation 
type, or another particular type of surface, e.g. paved areas or water.

If you choose another topic, I hope knowing how to load and visualise data is still useful - it's a fundamental skill!